In [1]:
import numpy as np # data in a vectorized manner manipulation
import pandas as pd # data analsysis
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # JSON files manipulation
import requests # HTTP library
from bs4 import BeautifulSoup # scraping library

from sklearn.cluster import KMeans # clustering algorithm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
# Download csv
# !wget -O landregistry.csv http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv

In [3]:
# Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("./pp-2018.csv")


# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

df_ppd.head()

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
0,{7E86B6FA-70B8-458C-E053-6B04A8C0C84C},355000,2018-10-19 00:00,UB3 1DZ,F,Y,L,"BOILER HOUSE, 2",FLAT 54,MATERIAL WALK,NaN,HAYES,HILLINGDON,GREATER LONDON,A,A
1,{7E86B6FA-70B9-458C-E053-6B04A8C0C84C},465000,2018-09-14 00:00,EN5 2FQ,F,Y,L,"DELPHI HOUSE, 4",FLAT 5,HERA AVENUE,NaN,BARNET,BARNET,GREATER LONDON,A,A
2,{7E86B6FA-70BA-458C-E053-6B04A8C0C84C},540000,2018-09-14 00:00,EN5 2FQ,F,Y,L,"DELPHI HOUSE, 4",FLAT 17,HERA AVENUE,NaN,BARNET,BARNET,GREATER LONDON,A,A
3,{7E86B6FA-70BB-458C-E053-6B04A8C0C84C},415000,2018-10-02 00:00,N13 5EX,F,Y,L,"HAZELTREE LODGE, 16 - 18",FLAT 9,HAZELWOOD LANE,NaN,LONDON,ENFIELD,GREATER LONDON,A,A
4,{7E86B6FA-70BC-458C-E053-6B04A8C0C84C},470000,2018-09-17 00:00,EN5 2FQ,F,Y,L,"DELPHI HOUSE, 4",FLAT 21,HERA AVENUE,NaN,BARNET,BARNET,GREATER LONDON,A,A


In [4]:
# Format the date column
print(df_ppd.dtypes)

TUID             object
Price             int64
Date_Transfer    object
Postcode         object
Prop_Type        object
Old_New          object
Duration         object
PAON             object
SAON             object
Street           object
Locality         object
Town_City        object
District         object
County           object
PPD_Cat_Type     object
Record_Status    object
dtype: object


In [5]:
df_ppd_city = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_city['Street'].unique().tolist()

In [6]:
df_grp_price = df_ppd_city.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [7]:
df_affordable = df_grp_price.query("(Avg_Price >= 2100000) & (Avg_Price <= 2200000)")
df_affordable.head()

,Street,Avg_Price
547,ASHCHURCH PARK VILLAS,2.150000e+06
665,AVENUE ROAD,2.143471e+06
753,BALLINGDON ROAD,2.105000e+06
1123,BERESFORD TERRACE,2.100000e+06
1422,BOSTON PLACE,2.167500e+06


In [8]:
from geopy.geocoders import Nominatim

nom = Nominatim(user_agent="my-application")

In [9]:
df_temp = pd.DataFrame()
df_temp['Full Address'] = df_affordable['Street'] + ', London'
df_temp['Full Address coords'] = df_temp['Full Address'].apply(nom.geocode)
df_temp.head()

,Full Address,Full Address coords
547,"ASHCHURCH PARK VILLAS, London","(Ashchurch Park Villas, Brook Green, London Bo..."
665,"AVENUE ROAD, London","(Avenue Road, Mackenzie Road, Penge, Bromley, ..."
753,"BALLINGDON ROAD, London","(Ballingdon Road, Balham, London Borough of Wa..."
1123,"BERESFORD TERRACE, London","(Beresford Terrace, Mildmay Park, Canonbury, L..."
1422,"BOSTON PLACE, London","(Boston Place, Marylebone, City of Westminster..."


In [10]:

df_temp['Latitude'] = df_temp['Full Address coords'].apply(lambda x : x.latitude if x != None else None)
df_temp['Longitude'] = df_temp['Full Address coords'].apply(lambda x : x.longitude if x != None else None)
df_temp.head()

,Full Address,Full Address coords,Latitude,Longitude
547,"ASHCHURCH PARK VILLAS, London","(Ashchurch Park Villas, Brook Green, London Bo...",51.500051,-0.242173
665,"AVENUE ROAD, London","(Avenue Road, Mackenzie Road, Penge, Bromley, ...",51.406797,-0.049519
753,"BALLINGDON ROAD, London","(Ballingdon Road, Balham, London Borough of Wa...",51.454189,-0.158856
1123,"BERESFORD TERRACE, London","(Beresford Terrace, Mildmay Park, Canonbury, L...",51.550294,-0.091434
1422,"BOSTON PLACE, London","(Boston Place, Marylebone, City of Westminster...",51.523997,-0.162951


In [11]:
# df_affordable.join(df_temp)
final = pd.concat([df_affordable, df_temp], axis=1, sort=False)
final = final.dropna()
print(final.dtypes)
final.head()

Street                  object
Avg_Price              float64
Full Address            object
Full Address coords     object
Latitude               float64
Longitude              float64
dtype: object


,Street,Avg_Price,Full Address,Full Address coords,Latitude,Longitude
547,ASHCHURCH PARK VILLAS,2.150000e+06,"ASHCHURCH PARK VILLAS, London","(Ashchurch Park Villas, Brook Green, London Bo...",51.500051,-0.242173
665,AVENUE ROAD,2.143471e+06,"AVENUE ROAD, London","(Avenue Road, Mackenzie Road, Penge, Bromley, ...",51.406797,-0.049519
753,BALLINGDON ROAD,2.105000e+06,"BALLINGDON ROAD, London","(Ballingdon Road, Balham, London Borough of Wa...",51.454189,-0.158856
1123,BERESFORD TERRACE,2.100000e+06,"BERESFORD TERRACE, London","(Beresford Terrace, Mildmay Park, Canonbury, L...",51.550294,-0.091434
1422,BOSTON PLACE,2.167500e+06,"BOSTON PLACE, London","(Boston Place, Marylebone, City of Westminster...",51.523997,-0.162951


In [12]:
city_loc = nom.geocode('London, UK')
print(city_loc.latitude, city_loc.longitude)

51.5073219 -0.1276474


In [16]:
map_london = folium.Map(location=[51.5073219, -0.1276474], zoom_start=11)

for location in final.itertuples(): #iterate each row of the dataframe
    label = 'Street: {};  Average Price: {};'.format(location[1], location[2])
    label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [location[-2], location[-1]],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)
    
    
    
map_london

In [17]:
CLIENT_ID = 'Y2SDCM5ONE2XYX5RQPDJSYNJCNM3BQVGHF0Z0OCRDCWXKNZC' # your Foursquare ID
CLIENT_SECRET = 'QA4IVQZIZ4TLH4BZTZKWE3Y1JKGVL12SHMDE2IBP43VM45ZS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Your credentails:
CLIENT_ID: Y2SDCM5ONE2XYX5RQPDJSYNJCNM3BQVGHF0Z0OCRDCWXKNZC
CLIENT_SECRET:QA4IVQZIZ4TLH4BZTZKWE3Y1JKGVL12SHMDE2IBP43VM45ZS


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
location_venues = getNearbyVenues(names=final['Street'],
                                   latitudes=final['Latitude'],
                                   longitudes=final['Longitude']
                                  )



ASHCHURCH PARK VILLAS
AVENUE ROAD
BALLINGDON ROAD
BERESFORD TERRACE
BOSTON PLACE
BRACKENBURY GARDENS
BRAMSHOT AVENUE
BROOKFIELD PARK
BROWNING CLOSE
BRYANSTON SQUARE
CANFIELD GARDENS
CARLISLE ROAD
CARLYLE CLOSE
CHANCE STREET
CLEVELAND SQUARE
COLINETTE ROAD
COLLEGE CROSS
COTSWOLD MEWS
CRANLEY MEWS
CUMBERLAND TERRACE
DIANA ROAD
DRYBURGH ROAD
DUCHESS WALK
ECCLESTON MEWS
EGERTON PLACE
FLORAL STREET
GREAT RUSSELL STREET
GREEN CLOSE
GROSVENOR GARDENS
HAMBLEDON PLACE
HENDERSON ROAD
HEWER STREET
HIGHLEVER ROAD
HILLGATE PLACE
HOBHOUSE COURT
IVOR PLACE
LEINSTER MEWS
LONG LANE
MANSON MEWS
MARGIN DRIVE
MAUNSEL STREET
MEADOWBANK
MILFORD LANE
MOLYNEUX STREET
MOORGATE
MUSEUM STREET
ONSLOW CRESCENT
PARKFIELDS
PAVILION ROAD
PENCOMBE MEWS
PLAYHOUSE YARD
PUTNEY HEATH LANE
QUEENS GATE GARDENS
RACTON ROAD
RANDOLPH MEWS
SHELDON SQUARE
ST JAMES'S STREET
STAFFORD STREET
TANNER STREET
TIERNEY LANE
TITE STREET
WINTERBROOK ROAD
WOODBOROUGH ROAD


In [23]:
location_venues.head()

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ASHCHURCH PARK VILLAS,51.500051,-0.242173,Detour Café,51.502391,-0.242705,Coffee Shop
1,ASHCHURCH PARK VILLAS,51.500051,-0.242173,The Eagle,51.500600,-0.239431,Pub
2,ASHCHURCH PARK VILLAS,51.500051,-0.242173,Som Tam House,51.502508,-0.242832,Thai Restaurant
3,ASHCHURCH PARK VILLAS,51.500051,-0.242173,Laveli Bakery,51.503074,-0.243160,Bakery
4,ASHCHURCH PARK VILLAS,51.500051,-0.242173,Ravenscourt Park,51.496614,-0.238652,Park


In [24]:
print('{} venues were returned.'.format(location_venues.shape[0]))
print('there were {} unique venue categories'.format(len(location_venues['Venue Category'].unique())))

3031 venues were returned.
there were 267 unique venue categories


In [29]:
venues_in_each = location_venues.groupby('Street').count()
venues_in_each = pd.DataFrame(list(zip(venues_in_each.index, venues_in_each['Venue'])), columns=['Street', 'num of Venues']).set_index('Street').join(final[['Street']].set_index('Street'), on='Street').reset_index()
# venues_in_each['Distance'] = np.int64(venues_in_each['Distance'])

print(venues_in_each.head())

venues_in_each[['num of Venues']].describe()

                  Street  num of Venues
0  ASHCHURCH PARK VILLAS             27
1            AVENUE ROAD              4
2        BALLINGDON ROAD             12
3      BERESFORD TERRACE             28
4           BOSTON PLACE             89


,num of Venues
count,63.000000
mean,48.111111
std,37.193960
min,3.000000
25%,12.500000
50%,39.000000
75%,91.000000
max,100.000000


In [30]:
# one hot encoding
streets_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
streets_onehot['Street'] = location_venues['Street'] 

# move neighborhood column to the first column
fixed_columns = [streets_onehot.columns[-1]] + list(streets_onehot.columns[:-1])
streets_onehot = streets_onehot[fixed_columns]

streets_onehot.head()

,Street,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Camera Store,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Donut Shop,Dry Cleaner,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Iraqi Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Event Space,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Pakistani Restaurant,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Roof Deck,Rugby Pitch,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Scottish Restaurant,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soba Restaurant,Soccer Field,Social Club,South American Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tour Provider,Toy / Game Store,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,ASHCHURCH PARK VILLAS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [32]:
streets_grouped = streets_onehot.groupby('Street').mean().reset_index()
print(streets_grouped.shape)
streets_grouped

(63, 268)


,Street,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Camera Store,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Donut Shop,Dry Cleaner,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Iraqi Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Event Space,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Pakistani Restaurant,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Roof Deck,Rugby Pitch,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Scottish Restaurant,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soba Restaurant,Soccer Field,Social Club,South American Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tour Provider,Toy / Game Store,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,ASHCHURCH PARK VILLAS,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.

In [36]:
num_top_venues = 2

for hood in streets_grouped['Street']:
    print("----"+hood+"----")
    temp = streets_grouped[streets_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ASHCHURCH PARK VILLAS----
           venue  freq
0  Grocery Store  0.15
1    Coffee Shop  0.11


----AVENUE ROAD----
           venue  freq
0           Park  0.25
1  Grocery Store  0.25


----BALLINGDON ROAD----
  venue  freq
0   Pub  0.25
1  Café  0.25


----BERESFORD TERRACE----
        venue  freq
0         Pub  0.18
1  Restaurant  0.14


----BOSTON PLACE----
         venue  freq
0         Café  0.07
1  Coffee Shop  0.07


----BRACKENBURY GARDENS----
         venue  freq
0  Coffee Shop  0.12
1          Pub  0.12


----BRAMSHOT AVENUE----
      venue  freq
0  Bus Stop  0.33
1       Spa  0.17


----BROOKFIELD PARK----
                      venue  freq
0                       Pub  0.20
1  Mediterranean Restaurant  0.13


----BROWNING CLOSE----
                        venue  freq
0                Home Service  0.17
1  Construction & Landscaping  0.17


----BRYANSTON SQUARE----
                       venue  freq
0                      Hotel  0.10
1  Middle Eastern Restaurant  0.06




In [38]:
# set number of clusters
kclusters = 5

streets_grouped_clustering = streets_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(streets_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:38] 

array([4, 2, 4, 4, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 2, 0, 0, 0, 0, 4, 2,
       0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 2, 4, 0, 0, 0, 4], dtype=int32)

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Street'] = streets_grouped['Street']

for ind in np.arange(streets_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(streets_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,ASHCHURCH PARK VILLAS,Grocery Store,Coffee Shop,Pub,Park,Indian Restaurant
1,AVENUE ROAD,Grocery Store,Park,Tapas Restaurant,Tram Station,Farmers Market
2,BALLINGDON ROAD,Pub,Café,Accessories Store,Italian Restaurant,Bakery
3,BERESFORD TERRACE,Pub,Restaurant,Café,Pizza Place,Turkish Restaurant
4,BOSTON PLACE,Café,Coffee Shop,Pub,Hotel,Grocery Store


In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

streets_merged = final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
streets_merged = streets_merged.join(neighborhoods_venues_sorted.set_index('Street'), on='Street')

streets_merged

,Street,Avg_Price,Full Address,Full Address coords,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
547,ASHCHURCH PARK VILLAS,2.150000e+06,"ASHCHURCH PARK VILLAS, London","(Ashchurch Park Villas, Brook Green, London Bo...",51.500051,-0.242173,4,Grocery Store,Coffee Shop,Pub,Park,Indian Restaurant
665,AVENUE ROAD,2.143471e+06,"AVENUE ROAD, London","(Avenue Road, Mackenzie Road, Penge, Bromley, ...",51.406797,-0.049519,2,Grocery Store,Park,Tapas Restaurant,Tram Station,Farmers Market
753,BALLINGDON ROAD,2.105000e+06,"BALLINGDON ROAD, London","(Ballingdon Road, Balham, London Borough of Wa...",51.454189,-0.158856,4,Pub,Café,Accessories Store,Italian Restaurant,Bakery
1123,BERESFORD TERRACE,2.100000e+06,"BERESFORD TERRACE, London","(Beresford Terrace, Mildmay Park, Canonbury, L...",51.550294,-0.091434,4,Pub,Restaurant,Café,Pizza Place,Turkish Restaurant
1422,BOSTON PLACE,2.167500e+06,"BOSTON PLACE, London","(Boston Place, Marylebone, City of Westminster...",51.523997,-0.162951,0,Café,Coffee Shop,Pub,Hotel,Grocery Store
1502,BRACKENBURY GARDENS,2.150000e+06,"BRACKENBURY GARDENS, London","(Brackenbury Gardens, Brook Green, London Boro...",51.500623,-0.230729,0,Chinese Restaurant,Pub,Grocery Store,Coffee Shop,Tea Room
1546,BRAMSHOT AVENUE,2.177900e+06,"BRAMSHOT AVENUE, London","(Bramshot Avenue, East Greenwich, Greenwich, L...",51.481160,0.022652,0,Bus Stop,Spa,Rugby Pitch,Train Station,Fast Food Restaurant
1762,BROOKFIELD PARK,2.150000e+06,"BROOKFIELD PARK, London","(Brookfield Park, Tufnell Park, London Borough...",51.561811,-0.146356,4,Pub,Mediterranean Restaurant,Grocery Store,Farmers Market,Food
1810,BROWNING CLOSE,2.160000e+06,"BROWNING CLOSE, London","(Browning Close, Collier Row, London Borough o...",51.599607,0.149130,0,Massage Studio,Gym,Construction & Landscaping,Home Service,Print Shop
1854,BRYANSTON SQUARE,2.197583e+06,"BRYANSTON SQUARE, London","(Bryanston Square, Marylebone, City of Westmin...",51.517067,-0.160365,0,Hotel,Middle Eastern Restaurant,Lebanese Restaurant,Sandwich Place,Restaurant


In [43]:
streets_merged.loc[streets_merged['Cluster Labels'] == 0, streets_merged.columns[[1] + list(range(5, streets_merged.shape[1]))]]

,Avg_Price,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1422,2.167500e+06,-0.162951,0,Café,Coffee Shop,Pub,Hotel,Grocery Store
1502,2.150000e+06,-0.230729,0,Chinese Restaurant,Pub,Grocery Store,Coffee Shop,Tea Room
1546,2.177900e+06,0.022652,0,Bus Stop,Spa,Rugby Pitch,Train Station,Fast Food Restaurant
1810,2.160000e+06,0.149130,0,Massage Studio,Gym,Construction & Landscaping,Home Service,Print Shop
1854,2.197583e+06,-0.160365,0,Hotel,Middle Eastern Restaurant,Lebanese Restaurant,Sandwich Place,Restaurant
2137,2.188333e+06,-0.179709,0,Coffee Shop,Café,Italian Restaurant,Pizza Place,Grocery Store
2227,2.175000e+06,-0.258852,0,Hotel,Pub,Middle Eastern Restaurant,Chinese Restaurant,Bed & Breakfast
2421,2.177000e+06,-0.075103,0,Restaurant,Pub,Pizza Place,Coffee Shop,Café
2839,2.150000e+06,-0.183167,0,Hotel,Pub,Café,Grocery Store,Coffee Shop
2978,2.185000e+06,-0.105443,0,Pub,Park,Mediterranean Restaurant,Cocktail Bar,French Restaurant


In [44]:
streets_merged.loc[streets_merged['Cluster Labels'] == 1, streets_merged.columns[[1] + list(range(5, streets_merged.shape[1]))]]

,Avg_Price,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12552,2120500.0,-73.323612,1,Food Truck,Botanical Garden,Flea Market,Falafel Restaurant,Farm


In [45]:
streets_merged.loc[streets_merged['Cluster Labels'] == 2, streets_merged.columns[[1] + list(range(5, streets_merged.shape[1]))]]

,Avg_Price,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
665,2.143471e+06,-0.049519,2,Grocery Store,Park,Tapas Restaurant,Tram Station,Farmers Market
2970,2.124375e+06,-0.229843,2,Convenience Store,Grocery Store,Tennis Court,Bus Stop,Café
4007,2.165625e+06,-0.230559,2,Convenience Store,Coffee Shop,Park,Gastropub,Café
6148,2.146000e+06,0.031668,2,Grocery Store,Fast Food Restaurant,Ice Cream Shop,Bus Stop,Pub
6318,2.143333e+06,-0.224509,2,Park,Playground,Soccer Field,Nature Preserve,Grocery Store
9589,2.200000e+06,-0.037249,2,Indian Restaurant,Antique Shop,Fast Food Restaurant,Grocery Store,Food Court
10154,2.150000e+06,-0.221080,2,Park,Bus Station,Bus Stop,Wine Shop,Intersection
13821,2.125000e+06,-0.234834,2,Grocery Store,Convenience Store,Park,Indian Restaurant,Italian Restaurant


In [46]:
streets_merged.loc[streets_merged['Cluster Labels'] == 3, streets_merged.columns[[1] + list(range(5, streets_merged.shape[1]))]]

,Avg_Price,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5565,2.102667e+06,-0.182441,3,Cosmetics Shop,Fruit & Vegetable Store,Photography Lab,Field,Zoo Exhibit


In [47]:
streets_merged.loc[streets_merged['Cluster Labels'] == 4, streets_merged.columns[[1] + list(range(5, streets_merged.shape[1]))]]

,Avg_Price,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
547,2.150000e+06,-0.242173,4,Grocery Store,Coffee Shop,Pub,Park,Indian Restaurant
753,2.105000e+06,-0.158856,4,Pub,Café,Accessories Store,Italian Restaurant,Bakery
1123,2.100000e+06,-0.091434,4,Pub,Restaurant,Café,Pizza Place,Turkish Restaurant
1762,2.150000e+06,-0.146356,4,Pub,Mediterranean Restaurant,Grocery Store,Farmers Market,Food
2211,2.200000e+06,-0.209667,4,Café,Coffee Shop,Park,Pub,Zoo Exhibit
3832,2.125000e+06,-0.023588,4,Café,Grocery Store,Pub,Social Club,Park
6359,2.200000e+06,-0.197397,4,Pub,Bakery,Park,Sushi Restaurant,Coffee Shop
7791,2.139212e+06,-0.088216,4,Pub,Coffee Shop,Italian Restaurant,Café,Restaurant
8180,2.100000e+06,-0.219175,4,French Restaurant,Pub,Thai Restaurant,Coffee Shop,Gastropub
8380,2.145833e+06,-0.160155,4,Pub,Café,Coffee Shop,Italian Restaurant,Pet Store


In [55]:
import math

# create map
map_clusters = folium.Map(location=[city_loc.latitude, city_loc.longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, .2, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(streets_merged['Latitude'], streets_merged['Longitude'], streets_merged['Street'], streets_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=1).add_to(map_clusters)


map_clusters